<a href="https://colab.research.google.com/github/Mperorrc/Semi-Supervised-Double-Deep-Q-Networks/blob/main/dqn_using_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from gym import Env
from gym.spaces import Discrete, Box

In [ ]:
# !pip install tensorflow==2.11.0
# !pip install keras==2.11.0
# !pip install keras-rl2
# !pip install rl
# !pip install tf-agents

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.11.0
2.11.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DDQN/dataset.xlsx')

In [ ]:
df['ABP2'] = df['ABP2'].fillna(0)

In [ ]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,SpO2,HR,PULSE,RESP,ABP2,ABP3,ALERT
0,94.0,71.000000,71.000000,19.0,113.000000,50.000000,0
1,94.0,71.000000,71.000000,19.0,113.000000,50.000000,0
2,94.0,71.000000,71.000000,17.0,113.000000,51.000000,0
3,94.0,71.000000,71.000000,17.0,113.000000,51.000000,0
4,94.0,71.000000,71.000000,17.0,113.000000,50.000000,0
...,...,...,...,...,...,...,...
253021,94.0,111.000000,111.000000,14.0,107.000000,61.000000,0
253022,94.0,111.000000,110.000000,14.0,108.000000,61.000000,0
253023,94.0,111.000000,110.000000,14.0,108.000000,61.000000,0
253024,94.0,111.000000,110.000000,14.0,108.000000,61.000000,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
scaler = MinMaxScaler()
columns = ['SpO2','HR','PULSE','RESP','ABP2','ABP3']
X = pd.DataFrame(scaler.fit_transform(df[['SpO2','HR','PULSE','RESP','ABP2','ABP3']]), columns=columns)
Y = df['ALERT']

In [ ]:
split_index = int(0.7 * len(X))

# Splitting the data into training and testing sets
x_train, x_test = X[:split_index], X[split_index:]
y_train, y_test = Y[:split_index], Y[split_index:]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
x_train

,SpO2,HR,PULSE,RESP,ABP2,ABP3
0,0.94,0.458065,0.510791,0.038,0.323288,0.209302
1,0.94,0.458065,0.510791,0.038,0.323288,0.209302
2,0.94,0.458065,0.510791,0.034,0.323288,0.211886
3,0.94,0.458065,0.510791,0.034,0.323288,0.211886
4,0.94,0.458065,0.510791,0.034,0.323288,0.209302
...,...,...,...,...,...,...
177113,0.00,0.606452,0.000000,0.016,0.276712,0.219638
177114,0.00,0.612903,0.000000,0.016,0.279452,0.219638
177115,0.00,0.612903,0.000000,0.016,0.282192,0.222222
177116,0.00,0.612903,0.000000,0.016,0.282192,0.222222


In [ ]:
x_train.loc[x_train.isnull().any(axis=1), 'ALERT'] = 1

# Replace NaN values with zeros in the entire DataFrame
x_train.fillna(0, inplace=True)
nan_indices = np.isnan(x_train)
nan_count = np.sum(nan_indices)
print(nan_count)


SpO2     0
HR       0
PULSE    0
RESP     0
ABP2     0
ABP3     0
ALERT    0
dtype: int64


<ipython-input-59-02b4f2c0c199>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.loc[x_train.isnull().any(axis=1), 'ALERT'] = 1
<ipython-input-59-02b4f2c0c199>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.fillna(0, inplace=True)


In [ ]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
177113    1
177114    1
177115    1
177116    1
177117    1
Name: ALERT, Length: 177118, dtype: int64

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform', input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3, activation='linear'))
model.compile(loss="mse", optimizer = tf.keras.optimizers.Adam(learning_rate=0.1))

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=200, shuffle=False)

Epoch 1/100
886/886 [==============================] - 20s 21ms/step - loss: 4.9961
Epoch 2/100
886/886 [==============================] - 17s 19ms/step - loss: 0.1707
Epoch 3/100
886/886 [==============================] - 18s 20ms/step - loss: 0.1347
Epoch 4/100
886/886 [==============================] - 19s 21ms/step - loss: 0.1999
Epoch 5/100
886/886 [==============================] - 17s 19ms/step - loss: 0.1785
Epoch 6/100
464/886 [==============>...............] - ETA: 9s - loss: 0.1450

KeyboardInterrupt: 

In [ ]:
model.get_weights()

[array([[ 9.7827077e-01,  8.3687454e-01, -3.1445226e-01,  3.3231878e-01,
          1.2135670e+00, -4.7259378e-01,  3.5693377e-01, -2.5031972e-01,
         -7.1440101e-01, -1.0815692e-01, -1.5282196e-01, -4.2236128e+00,
          1.1724235e+00,  3.1201833e-01, -3.7240577e-01, -7.9962444e-01,
         -6.3483119e-01, -5.8349401e-01, -1.3487974e+00, -1.2473780e+00,
         -7.2183609e-01, -1.0307450e+00, -1.2402753e+00,  1.1744138e+00,
          7.5210756e-01,  3.9296844e+00, -6.1249828e-01, -5.5637360e-01,
         -9.6342564e-03,  9.3121417e-02,  8.3874601e-01,  2.3590371e-01,
         -3.0709523e-01,  2.9365250e-01, -1.2061309e+00, -1.3599634e-02,
          1.1853189e+00, -6.6695881e-01,  4.3887767e-01,  5.1950234e-01,
          2.1920719e+00, -9.6879053e-01, -4.9307424e-01,  4.9331567e-01,
         -1.6885971e+00, -4.3007898e-01, -3.8920250e+00, -8.3534968e-01,
          1.2239305e+00,  1.0662055e-01, -1.2082852e+00, -4.3157658e-01,
          1.4936638e+00, -1.7600927e+00,  2.0350523

In [ ]:
# class DQN:
#   def __init__(self,learning_rate,gamma, runs):
#     self.gamma = gamma                                       # Discount rate for bellman equation
#     self.epsilon = 1                                         # parameter in epsilon greedy approach
#     self.epsilon_min = 0.01
#     self.epsilon_decay = 0.999
#     self.runs = runs                                         # number of simulations
#     self.input_dims = x_train.shape[1]                       # input dimensions
#     self.actions = len(y_train.unique())                     # 3 actions -> true alarm, false alarm, no alarm
#     self.replay_buffer = 1000                                # Experience replay queue size
#     self.training_buffer = 300                               # Training experience replay queue size
#     self.learning_rate = learning_rate                       # Alpha

# # Off policy Temporal Difference(0) Q learning
# # Q(sk,ak) = Q_{old}(sk,ak) + alpha(reward - Q_{old}(sk,ak))
# # reward = rk + gamma*max_{a}(Q(sk+1,a))



In [ ]:
# HYPERPARAMTERS

gamma = 0.99                         # Discount Rate
batch_size = 12
buffer_size = 10000
min_replay_size = 1000
epsilon_start = 1.0
epsilon_end = 0.02
#epsilon decay is the decay period in which it goes from eps_start to eps_end
epsilon_decay = 0.999
target_update_freq = 1000

# Custom environment

In [ ]:
class AnomalyEnv(Env):

    def __init__(self,X_train,y_train):
        self.x_train = X_train
        self.y_train = y_train
        self.action_space = Discrete(len(self.y_train.unique()))
        self.observation_space = Box(low=np.zeros(self.x_train.shape[1]), high=np.ones(self.x_train.shape[1]))
        self.index = 0
        self.x_train = np.array(self.x_train)
        self.state = self.x_train[0]
        print(self.state.shape,self.observation_space.shape)

    def step(self, action):
        self.state = self.x_train[self.index]
        reward = 0

        if action == y_train[self.index]:
            if action==0:
              reward = 1
            elif action == 1:
              reward = 5
            else:
              reward = 10
        else:
            if y_train[self.index]==0:
              reward = -1
            elif y_train[self.index] == 1:
              reward = -5
            else:
              reward = -10

        self.index += 1
        info = {}

        if self.index == len(self.x_train):
            done = True
        else:
            done = False
            self.state = self.x_train[self.index]

        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        self.index=0
        self.state=self.x_train[0]
        return self.state

In [ ]:
env = AnomalyEnv(x_train,y_train)

(6,) (6,)


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [ ]:
env.observation_space.sample()

array([0.8271753 , 0.5913803 , 0.21266064, 0.62067777, 0.67486507,
       0.03296787], dtype=float32)

In [ ]:
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-88589
Episode:2 Score:-89371


# DQN architecture

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.optimizers import Adam

In [ ]:
state = env.observation_space.shape
actions = env.action_space.n

In [ ]:
print(state)

(6,)


In [ ]:
def DQN_model(states,actions):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform',input_shape=(1,6)))
  model.add(tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_uniform'))
  model.add(tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_uniform'))
  model.add(tf.keras.layers.Dense(actions,activation='linear'))
  model.add(tf.keras.layers.Flatten())
  return model


In [ ]:
model = DQN_model(state,actions)
model.summary()

NameError: name 'state' is not defined

In [ ]:
from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
def agent(model, actions):
  policy = EpsGreedyQPolicy(eps=epsilon_start)
  memory = SequentialMemory(limit = 50000, window_length = 1)
  dqn = DQNAgent(model = model, policy = policy, memory = memory, nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
  return dqn

In [ ]:
dqn = agent(model,actions)


In [ ]:
# del model

In [ ]:
dqn.compile(Adam(learning_rate=1e-3),metrics=['mse'])

In [ ]:
dqn.fit(env,nb_steps=x_train.shape[0],visualize =False, verbose = 1)

Training for 177118 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 1:29:53 - reward: -1.0000

/usr/local/lib/python3.10/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   24/10000 [..............................] - ETA: 18:49 - reward: -0.5000

/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch

   36/10000 [..............................] - ETA: 13:13 - reward: -0.3333

/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 25 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 26 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 27 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 28 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.10/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 29 + 1) instead
  batch

10000/10000 [==============================] - 185s 18ms/step - reward: -0.4203
Interval 2 (10000 steps performed)
10000/10000 [==============================] - 184s 18ms/step - reward: -0.5761
Interval 3 (20000 steps performed)
10000/10000 [==============================] - 185s 19ms/step - reward: -0.4644
Interval 4 (30000 steps performed)
10000/10000 [==============================] - 185s 19ms/step - reward: -0.4450
Interval 5 (40000 steps performed)
10000/10000 [==============================] - 192s 19ms/step - reward: -0.3949
Interval 6 (50000 steps performed)
10000/10000 [==============================] - 195s 19ms/step - reward: -0.4549
Interval 7 (60000 steps performed)
10000/10000 [==============================] - 191s 19ms/step - reward: -0.5576
Interval 8 (70000 steps performed)
10000/10000 [==============================] - 192s 19ms/step - reward: -0.3347
Interval 9 (80000 steps performed)
10000/10000 [==============================] - 191s 19ms/step - reward: -0.3753


In [ ]:
predictions = []
for i in range(x_test.shape[0]):
    pred = model.predict(np.expand_dims(np.expand_dims(x_test.iloc[i], axis=0),axis=0))
    pred =np.argmax(pred)
    predictions.append(pred)



In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.935579912525689


In [ ]:
cor=0
tot=0
for i in range(y_test.shape[0]):
  if y_test.iloc[i]!=0:
    tot+=1
    if y_test.iloc[i]==predictions[i]:
      cor+=1
print(cor/tot)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.0


In [ ]:
cor=0
tot=0
for i in range(y_test.shape[0]):
  if y_test.iloc[i]==0:
    tot+=1
    if y_test.iloc[i]==predictions[i]:
      cor+=1
print(cor/tot)

1.0
